In [ ]:
import gurobipy as gp
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math

## Generazione dataset

Prima genera tutti i CC e gli SP che non appartengono a cluster, sono distribuiti uniformemente nella mappa.
Poi seleziona i CC che saranno il centro dei cluster, per ognuno di essi genera le coordinate degli SP (uniformemente), nella regione inclusa fra +- 3 punti di distanza. Tutti i cluster hanno stesso numero di SP

In [ ]:
#la funzione prende in input: k dimensioni mappa, N_SP numero di source points, n_cluster numero di cluster 
#e cluster ratio percentuale di punti facenti parte di un cluster
#ritorna in output vettori numpy contenenti le coordinate dei punti
def data_generation(k, n_SP, n_CC, n_cluster, cluster_ratio):
    
    #definisco coordinate di tutti i CC e degli SP non facenti parte dei cluster
    P=np.random.randint(0, k+1, size=(n_CC, 2))  #genera il set dei CC
    S_no_cluster=np.random.randint(0, k+1, size=(n_SP-int(cluster_ratio*n_SP), 2))  #genera il set dei SP non facenti parte di un cluster

    #genero coordinate SP rimanenti
    cluster_index=random.sample(range(0,n_CC), int(n_CC*cluster_ratio)) #seleziona l'indice dei CC a caso che diventeranno i centri dei cluster
    S_cluster_list=[]  #creo lista vuota da popolare con i numpy array e poi concatenarli TROVA MODO PIU EFFICIENTE

    #per ognuno dei cluster genero casualmente uniformemente le coordinate (entro i 36 punti adiacenti) di 5 SP
    for i in cluster_index:
        #voglio considerare il contorno dei 36 vicini, quindi indici (i-3, j-3) (i+3, j+3)
        xmin=P[i][0]-3
        xmax=P[i][0]+3

        ymin=P[i][1]-3
        ymax=P[i][1]+3
    
        #considera ogni cluster con stesso numero di SP
        S_cluster_list.append(np.random.randint(low=[xmin, ymin], high=[xmax, ymax], size=(round(cluster_ratio*n_SP/n_cluster), 2)))

    #mette assieme il tutto
    S_cluster=np.concatenate(S_cluster_list) #crea array unico per i cluster
    S=np.concatenate((S_cluster, S_no_cluster))

    #genera posizione della fabbrica biodiesel
    biodiesel=np.random.randint(0, k+1, size=(1, 2))
    
    return P, S, biodiesel

In [ ]:
#funzione che date le coordinate di due punti (in array 2d numpy) ritorna la distanza euclidea arrotondata in integer
def distance(a, b):
    dist=round(math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2))
    return dist

In [ ]:
file = open("time.txt","a") #apre il file in cui scrivere i tempi

#per vari valori di n_SP, n_CC vengono risolte 5 istanze del problema, salvando tempo e qualità soluzione
for n_SP,n_CC in zip([100, 200, 300, 400, 500], [20, 40, 60, 80, 100]):

    k=1000  #dimensioni della mappa (prima test con k=1000, poi varialo per studio domansionale)
    n_cluster=int(0.5*n_CC)  #number of clusters
    cluster_ratio=0.5  #how many of the total nodes (SPs and CCs) fall within a cluster
    sol_time=[]
    sol_gap=[]
    for i in range(5):
        P, S, biodiesel=data_generation(k, n_SP, n_CC, n_cluster, cluster_ratio)

        #definizione e generazione delle altre costanti
        alpha=1  #costo per unità di distanza
        V=5 #numero massimo di veicoli, numero effettivamente utilizzato viene ottimizzato, 5 è una scelta 'sicura' probabilmente può essere ridotto per acellerare la soluzione 
        Q=24  #numero di cestini su ogni veicolo
        B=50  #capacità, in litri, dei bins sui veicoli
        b=10  #costo di un bin
        max_distance=300  #distanza massima fra un SP e un CC
        R=1800 #lungezza massima percorso per veicolo, se distanza fra 0,0 e 1000,1000 è 1414 allora 1800 non è sufficiente nel caso il centrp biodiesel venga generato in un angolo?
        P0=np.concatenate((P, biodiesel)) #set posizione biodiesel+CC (serve in calcoli veicoli)
        f=np.random.randint(50, 150+1, size=(n_CC))  #costo per aprire il CC i
        d_s=np.random.randint(1, 20+1, size=(len(S)))  #produzione settimanale in litri di olio del SP i
        C=np.random.randint(1, 5+1, size=(n_CC))  #numero di Bins nel CC i

        #P_s è il set dei CC entro la distanza di copertura dal SP s
        #quindi per ogni SP creo lista P_s contente indice dei CC compatibili, poi salvo in lista di liste il tutto
        P_s_list=[]  #inizializzo lista che conterrà i vari P_s
        for s in range(len(S)):
            P_s=[]  #inizializzo P_s
            for i in range(len(P)): 
                if (distance(S[s], P[i])<300):   #controlla se la distanza è minore di 300
                    P_s.append(i)  #se distanza e minore di 300 salva l'indice del CC compatibile
            P_s_list.append(P_s)  #aggiunge la lista alla lista totale


        #crea matrice delle distanze fra elementi di P0, serve per funzione obbiettivo e constraints
        c0=np.empty([len(P0), len(P0)])
        for i in range(len(P0)):
            for j in range(len(P0)):
                c0[i,j]=distance(P0[i], P0[j])
        
        #crea i set degli indici
        P_set=range(0, n_CC) #CC
        S_set=range(0, n_SP) #SP
        P0_set=range(0, n_CC+1)  #CC+centro biodiesel (ultimo elemento)
        V_set=range(0, V)  #veicoli

        oil_model=gp.Model() #crea il modello
        oil_model.modelSense = gp.GRB.MINIMIZE

        #decision variables
        #1 se il CC i viene aperto, 0 altrimenti
        x_i=oil_model.addVars([(i) for i in P_set], vtype=gp.GRB.BINARY) 

        #1 se il SP s viene assegnato al CC i, 0 altrimenti.
        z_si=oil_model.addVars([(s, i) for s in S_set for i in P_s_list[s]], vtype=gp.GRB.BINARY) 

        #1 se il veicolo k viaggia da i a j, 0 altrimenti
        v_ijk=oil_model.addVars([(i, j, k) for i in P0_set for j in P0_set for k in V_set], vtype=gp.GRB.BINARY) 

        #numero di bins raccolti dal veicolo k al CC i
        t_ik=oil_model.addVars([(i, k) for i in P_set for k in V_set], lb=0, vtype=gp.GRB.INTEGER) 

        #variabile ausiliaria per eliminazione dei subtours
        u_i=oil_model.addVars([(i) for i in P_set], lb=0, vtype=gp.GRB.INTEGER) 


        #objective function and constraints
        #1)objective function
        oil_model.setObjective(alpha*gp.quicksum(c0[i,j]*v_ijk[i,j,k] for i in P0_set for j in P0_set for k in V_set)+
                               gp.quicksum(f[i]*x_i[i] for i in P_set)+
                               b*gp.quicksum(t_ik[i,k] for i in P_set for k in V_set)
                              )

        #2)ensure that the total amount of WCO generated by the SPs assigned to a CC is not higher than the 
        #total capacity of the bins placed at that CC
        oil_model.addConstrs(gp.quicksum(d_s[s]*z_si[s, i] for s in S_set if i in P_s_list[s])<=
                             B*gp.quicksum(t_ik[i,k] for k in V_set)
                             for i in P_set 
                            )

        #3)limit the capacity of each CC in terms of the number of bins. These constraints also make sure that a CC must be opened if bins are
        #placed at that CC.
        oil_model.addConstrs(gp.quicksum(t_ik[i,k] for k in V_set)<=
                             C[i]*x_i[i]
                             for i in P_set 
                            )

        #4)force a CC to be opened if an SP is assigned to it.
        oil_model.addConstrs(z_si[s,i]<=
                             x_i[i]
                             for s in S_set for i in P_s_list[s] 
                            )    
        #5)Each SP is assigned to one of the CCs within the coverage distance
        oil_model.addConstrs(gp.quicksum(z_si[s,i] for i in P_s_list[s])==1 
                             for s in S_set  
                            )   

        #6)guarantee that each vehicle leaves the biodiesel facility at most once.
        #in questo caso il centro biodiesel è l'ultimo punto di P0, quindi questo constraint è leggermente diverso da quello nel paper
        oil_model.addConstrs(gp.quicksum(v_ijk[len(P0)-1,i,k] for i in P_set)<=1
                             for k in V_set 
                            )    

        #7)A collection vehicle can pick up bins at a CC only if it visits the CC
        oil_model.addConstrs(t_ik[i,k]<=
                             Q*gp.quicksum(v_ijk[j,i,k] for j in P0_set if j!=i)
                             for i in P_set for k in V_set
                            )   

        #8)total number of bins picked up by a vehicle cannot be greater than the vehicle capacity.
        oil_model.addConstrs(gp.quicksum(t_ik[i,k] for i in P_set)<=
                             Q
                             for k in V_set
                            )

        #9)route continuity constraints.
        oil_model.addConstrs(gp.quicksum(v_ijk[i,j,k] for j in P0_set)==
                             gp.quicksum(v_ijk[j,i,k] for j in P0_set)
                             for i in P0_set for k in V_set
                            )  

        #10)ensure that a vehicle is routed through a CC if and only if that CC is opened.
        oil_model.addConstrs(gp.quicksum(v_ijk[i,j,k] for j in P0_set for k in V_set if j!=i)==
                             x_i[i]
                             for i in P_set
                            )  

        #11)ensure that a vehicle is routed through a CC if and only if that CC is opened.
        oil_model.addConstrs(gp.quicksum(v_ijk[j,i,k] for j in P0_set for k in V_set if j!=i)==
                             x_i[i]
                             for i in P_set 
                            )  

        #12)eliminates subtours
        oil_model.addConstrs(u_i[i]-u_i[j]+len(P_set)*gp.quicksum(v_ijk[i,j,k] for k in V_set)<=
                             len(P_set)-1
                             for i in P_set for j in P_set if i!=j
                            )  

        #13)restrict the travel distance of each route by the maximum route duration.
        oil_model.addConstrs(gp.quicksum(c0[i,j]*v_ijk[i,j,k] for i in P0_set for j in P0_set)<=
                             R
                             for k in V_set
                            )  

        #14-18) sono gia definiti implicitamente nella inizializzazione delle decision variables

        #update del modello per aggiunger la funzione obbiettivo e i constraints
        oil_model.update()
    
        oil_model.setParam('TimeLimit', 7200)  #set tempo massimo ricerca soluzione...
        oil_model.optimize()
        file.write(str(n_CC)+" "+str(n_SP)+" "+str(oil_model.Runtime)+" "+str(oil_model.MIPGap)+"\n")
        sol_time.append(oil_model.Runtime)
        sol_gap.append(oil_model.MIPGap)
        oil_model.dispose()
        
file.close() #chiude il file